# Lab: APIs

In [29]:
#!pip install googleplaces

ERROR: Could not find a version that satisfies the requirement googleplaces (from versions: none)
ERROR: No matching distribution found for googleplaces


##### Instructions: 

Write your own Python script to answer the following questions: 
1. Which of these embassies is closest to the White House in meters? What is the address? 
2. If I wanted to hold a morning meeting there, which cafe would you suggest (best rating and closest)?
3. If I wanted to hold an upscale evening meeting there, which fancy bar would you suggest? 

Hint: 
- You will need to enable the `Google Places API`.
- If your request gets denied, make sure
    - 1. The API is enabled on your Google account
      2. Your credential is unrestricted to all APIs 
- You may find this page useful to learn about different findinging nearby places https://www.geeksforgeeks.org/python-fetch-nearest-hospital-locations-using-googlemaps-api/

In [30]:
import importlib
import os
import googlemaps
import geopy.distance

# os.chdir('you/key/dir')
imported_items = importlib.import_module('start_google')
gmaps = imported_items.client

whitehouse = '1600 Pennsylvania Avenue, Washington, DC'
white_house_coords = (38.8976763, -77.0365298)

# Coordinates of the embassies
embassies = [
    (38.917228, -77.0522365), 
    (38.9076502, -77.0370427), 
    (38.916944, -77.048739)
]


In [31]:


def calculate_distance(coords1, coords2):
    return geopy.distance.distance(coords1, coords2).kilometers

distances = [calculate_distance(white_house_coords, embassy) for embassy in embassies]

closest_embassy_index = distances.index(min(distances))
closest_embassy_coords = embassies[closest_embassy_index]
closest_distance = distances[closest_embassy_index]

# Print results
print(f"The closest embassy is located at coordinates {closest_embassy_coords} which is approximately {closest_distance:.2f} k away from the White House.")




The closest embassy is located at coordinates (38.9076502, -77.0370427) which is approximately 1.11 k away from the White House.


In [53]:
# Search for cafés near the White House

white_house_coords = (38.8976763, -77.0365298)

query_result = gmaps.places_nearby( 
        location = white_house_coords, 
        radius =5000, 
        type ='cafe')

cafes = query_result['results']

def calculate_distance(coords1, coords2):
    return geopy.distance.distance(coords1, coords2).meters # This calculates the distance bw coordinates

# Create a list of cafes with their distance from the White House and their rating
cafes_with_distance = []
for cafe in cafes:
    cafe_location = (cafe['geometry']['location']['lat'], cafe['geometry']['location']['lng'])
    distance = calculate_distance(white_house_coords, cafe_location) 
    rating = cafe.get('rating', 0)  # Some cafes might not have a rating
    cafes_with_distance.append({
        'name': cafe['name'],
        'address': cafe['vicinity'],
        'rating': rating,
        'distance': distance
    })

# Sort cafes by rating (descending) and then by distance (ascending)
best_cafes = sorted(cafes_with_distance, key=lambda x: (-x['rating'], x['distance']))

# Get the best-rated and closest cafe
best_cafe = best_cafes[0]

# Print the best cafe details
print(f"The best cafe for a morning meeting is '{best_cafe['name']}' located at {best_cafe['address']}.")
print(f"Rating: {best_cafe['rating']} | Distance: {best_cafe['distance']:.2f} meters")


The best cafe for a morning meeting is 'Le Diplomate' located at 1601 14th Street Northwest, Washington.
Rating: 4.6 | Distance: 1577.17 meters


In [47]:
whitehouse = (38.8976763, -77.0365298)

places_result = gmaps.places_nearby(
    location=whitehouse, 
    radius=500, 
    type='bar'
)

best_bar = None
highest_rating = 0

for place in places_result['results']:
    if 'rating' in place and place['rating'] > highest_rating:
        highest_rating = place['rating']
        best_bar = place

if best_bar:
    bar_name = best_bar['name']
    bar_address = best_bar['vicinity']
    bar_rating = best_bar['rating']
    
    distance = gmaps.distance_matrix(whitehouse, best_bar['geometry']['location'])
    distance_text2 = distance['rows'][0]['elements'][0]['distance']['text']
    
    print(f"Best Bar: {bar_name}")
    print(f"Address: {bar_address}")
    print(f"Rating: {bar_rating}")
    print(f"Distance from White House: {distance_text2}")
else:
    print("No suitable bars found nearby.")

Best Bar: Expat
Address: Western Market, 2000 Pennsylvania Avenue Northwest, Washington
Rating: 5
Distance from White House: 1.3 km
